In [1]:
import pandas as pd
from INEvalidador import Validador
from INEvalidador.conexionSQL import baseSQL
import re
import datetime
import os 

conexion_sql = baseSQL()

In [2]:
# Función que obtenga la primera variable en la validación para poder identificar qué pasa usará
def identificar_variable(condicion: str):
    variable = re.findall(r'^\s*([A-Za-z_][A-Za-z\d_]*)\s*[<>!=]*', condicion, re.MULTILINE)
    return variable

In [3]:
# Función para convertirlas todas las columnas de la base a mayuscula
def columnas_a_mayuscula(tabla:str):
    base = pd.read_feather(tabla)
    columnas_originales = base.columns
    columnas_nuevas = []
    for columna in columnas_originales:
        col = columna.upper()
        columnas_nuevas.append(col)
    diccionario = dict(zip(columnas_originales, columnas_nuevas))
    base = base.rename(columns=diccionario)
    return base

In [4]:
def clasificar_validaciones(condiciones:str, bases:list):
    validaciones = pd.read_excel(condiciones)
    df1 = columnas_a_mayuscula(bases[0])
    df2 = columnas_a_mayuscula(bases[1])
    df3 = columnas_a_mayuscula(bases[2])
    for validacion in list(validaciones["Condición o Criterio"]):
        # Asegurarse de que validacion sea una cadena
        validacion_str = str(validacion)
        var = identificar_variable(validacion_str)
        if var in list(df1.columns):
            validacion["Base_a_usar"] = 1
        if var in list(df2.columns):
            validacion["Base_a_usar"] = 2
        if var in list(df3.columns):
            validacion["Base_a_usar"] = 3
    for i in range(len(bases)):
        validaciones[validaciones["Base_a_usar"] == i].to_excel(f"ExpresionesBase{i}.xlsx")


In [8]:
def main(condiciones, bases: list=["Bases/Ronda1/personas.feather","Bases/Ronda1/HogaresRonda1.feather","Bases/Ronda2/HogaresRonda2.feather"]):
    conexion_sql.obtener_datos()
    clasificar_validaciones(condiciones,bases)
    marca_temp = datetime.now().strftime("%Y%m%d%H%M%S")
    carpeta_padre = "Inconsistencias_{}".format(marca_temp)
    for i in range(len(bases)):
        validador = Validador(bases[i],f'ExpresionesBase{i}.xlsx')
        validador.process_to_export(f'Parte{i}')
    dfs = []
    for i in range(len(bases)):
        dfs.append(pd.read_excel(f'ExpresionesBase{i}.xlsx'))
    df_exportacion = pd.concat(dfs)
    df_exportacion.to_excel(os.path.join(carpeta_padre, f'Inconsistencias.xlsx'),index=False)

In [10]:
main("Expresiones.xlsx")

KeyError: 'Base_a_usar'